In [51]:
# IMPORTAMOS LIBRERÍAS

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, MinMaxScaler

In [52]:
# LEEMOS EL FICHERO DE RENFE

df_bmw = pd.read_csv("bmw_pricing.csv", sep=",", encoding='latin-1')
df_bmw.head()

,marca,modelo,km,potencia,fecha_registro,tipo_gasolina,color,tipo_coche,volante_regulable,aire_acondicionado,camara_trasera,asientos_traseros_plegables,elevalunas_electrico,bluetooth,gps,alerta_lim_velocidad,precio,fecha_venta
0,NaN,118,140411.0,100.0,2012-02-01,diesel,black,NaN,True,True,False,NaN,True,NaN,True,NaN,11300.0,2018-01-01
1,BMW,M4,13929.0,317.0,NaN,petrol,grey,convertible,True,True,False,NaN,False,True,True,True,69700.0,2018-02-01
2,BMW,320,183297.0,120.0,2012-04-01,diesel,white,NaN,False,False,False,NaN,True,False,True,False,10200.0,2018-02-01
3,BMW,420,128035.0,135.0,NaN,diesel,red,convertible,True,True,False,NaN,True,True,True,NaN,25100.0,2018-02-01
4,BMW,425,97097.0,160.0,NaN,diesel,silver,NaN,True,True,False,False,False,True,True,True,33400.0,2018-04-01


In [53]:
df_bmw


,marca,modelo,km,potencia,fecha_registro,tipo_gasolina,color,tipo_coche,volante_regulable,aire_acondicionado,camara_trasera,asientos_traseros_plegables,elevalunas_electrico,bluetooth,gps,alerta_lim_velocidad,precio,fecha_venta
0,NaN,118,140411.0,100.0,2012-02-01,diesel,black,NaN,True,True,False,NaN,True,NaN,True,NaN,11300.0,2018-01-01
1,BMW,M4,13929.0,317.0,NaN,petrol,grey,convertible,True,True,False,NaN,False,True,True,True,69700.0,2018-02-01
2,BMW,320,183297.0,120.0,2012-04-01,diesel,white,NaN,False,False,False,NaN,True,False,True,False,10200.0,2018-02-01
3,BMW,420,128035.0,135.0,NaN,diesel,red,convertible,True,True,False,NaN,True,True,True,NaN,25100.0,2018-02-01
4,BMW,425,97097.0,160.0,NaN,diesel,silver,NaN,True,True,False,False,False,True,True,True,33400.0,2018-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4838,BMW,218 Gran Tourer,39743.0,110.0,NaN,diesel,black,NaN,False,True,False,NaN,False,False,True,False,14600.0,2018-08-01
4839,BMW,218 Active Tourer,49832.0,100.0,2015-06-01,diesel,grey,NaN,False,True,False,NaN,False,False,True,True,17500.0,2018-08-01
4840,BMW,218 Gran Tourer,19633.0,110.0,2015-10-01,diesel,grey,van,False,True,False,NaN,False,False,True,True,17000.0,2018-09-01
4841,BMW,218 Active Tourer,27920.0,110.0,2016-04-01,diesel,brown,van,True,True,False,False,False,False,True,True,22700.0,2018-09-01


In [54]:
df_bmw.describe()

,km,potencia,precio
count,4.841000e+03,4842.000000,4837.000000
mean,1.409593e+05,128.981826,15831.920612
std,6.020853e+04,38.994839,9222.630708
min,-6.400000e+01,0.000000,100.000000
25%,1.028840e+05,100.000000,10800.000000
50%,1.410800e+05,120.000000,14200.000000
75%,1.752170e+05,135.000000,18600.000000
max,1.000376e+06,423.000000,178500.000000


In [55]:
df_bmw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4843 entries, 0 to 4842
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   marca                        3873 non-null   object 
 1   modelo                       4840 non-null   object 
 2   km                           4841 non-null   float64
 3   potencia                     4842 non-null   float64
 4   fecha_registro               2420 non-null   object 
 5   tipo_gasolina                4838 non-null   object 
 6   color                        4398 non-null   object 
 7   tipo_coche                   3383 non-null   object 
 8   volante_regulable            4839 non-null   object 
 9   aire_acondicionado           4357 non-null   object 
 10  camara_trasera               4841 non-null   object 
 11  asientos_traseros_plegables  1452 non-null   object 
 12  elevalunas_electrico         4841 non-null   object 
 13  bluetooth         

In [56]:
df_bmw.nunique() # nos dice cuanta cantidad de valores distintos hay en cada columna

marca                             1
modelo                           76
km                             4784
potencia                         61
fecha_registro                  180
tipo_gasolina                     5
color                            10
tipo_coche                        8
volante_regulable                 2
aire_acondicionado                2
camara_trasera                    2
asientos_traseros_plegables       2
elevalunas_electrico              2
bluetooth                         2
gps                               2
alerta_lim_velocidad              2
precio                          437
fecha_venta                      13
dtype: int64

In [57]:
# fijamos que columna tiene valores nulos
df_bmw.isnull().any()


marca                           True
modelo                          True
km                              True
potencia                        True
fecha_registro                  True
tipo_gasolina                   True
color                           True
tipo_coche                      True
volante_regulable               True
aire_acondicionado              True
camara_trasera                  True
asientos_traseros_plegables     True
elevalunas_electrico            True
bluetooth                       True
gps                            False
alerta_lim_velocidad            True
precio                          True
fecha_venta                     True
dtype: bool

In [58]:
df_bmw[df_bmw["marca"].isnull()].shape

(970, 18)

In [59]:
df_bmw.isnull().sum()/df_bmw.shape[0]*100


marca                          20.028908
modelo                          0.061945
km                              0.041297
potencia                        0.020648
fecha_registro                 50.030973
tipo_gasolina                   0.103242
color                           9.188520
tipo_coche                     30.146603
volante_regulable               0.082593
aire_acondicionado             10.035102
camara_trasera                  0.041297
asientos_traseros_plegables    70.018584
elevalunas_electrico            0.041297
bluetooth                      15.032005
gps                             0.000000
alerta_lim_velocidad           15.032005
precio                          0.123890
fecha_venta                     0.020648
dtype: float64

In [60]:
modelos_nulos = df_bmw[df_bmw["marca"].isnull()]["modelo"].unique()
modelos_nulos_unicos = []

for i in modelos_nulos:
    registros_con_modelo = df_bmw[df_bmw["modelo"] == i]
    con_marca_repetidos = registros_con_modelo[registros_con_modelo["marca"].notnull()]
    if len(con_marca_repetidos) > 0:
        modelos_nulos_unicos.append(i)

modelos_nulos_unicos

['118',
 'Z4',
 '325',
 '218',
 '120',
 '425',
 '435',
 '640',
 '420',
 '220',
 'M4',
 '320',
 '318',
 '520',
 '316',
 '535',
 '525',
 '518',
 '530',
 'M550',
 '330',
 '116',
 '528',
 '328',
 '125',
 '420 Gran CoupÃ©',
 '114',
 '320 Gran Turismo',
 '318 Gran Turismo',
 '530 Gran Turismo',
 '418 Gran CoupÃ©',
 '520 Gran Turismo',
 '135',
 '435 Gran CoupÃ©',
 '640 Gran CoupÃ©',
 '740',
 '730',
 'X1',
 'X3',
 'X5',
 'X6',
 'X4',
 'X6 M',
 'X5 M',
 'X5 M50',
 '218 Active Tourer',
 '218 Gran Tourer']

In [61]:
 df_bmw.drop(columns=["marca"], inplace=True)


In [62]:
df_bmw


,modelo,km,potencia,fecha_registro,tipo_gasolina,color,tipo_coche,volante_regulable,aire_acondicionado,camara_trasera,asientos_traseros_plegables,elevalunas_electrico,bluetooth,gps,alerta_lim_velocidad,precio,fecha_venta
0,118,140411.0,100.0,2012-02-01,diesel,black,NaN,True,True,False,NaN,True,NaN,True,NaN,11300.0,2018-01-01
1,M4,13929.0,317.0,NaN,petrol,grey,convertible,True,True,False,NaN,False,True,True,True,69700.0,2018-02-01
2,320,183297.0,120.0,2012-04-01,diesel,white,NaN,False,False,False,NaN,True,False,True,False,10200.0,2018-02-01
3,420,128035.0,135.0,NaN,diesel,red,convertible,True,True,False,NaN,True,True,True,NaN,25100.0,2018-02-01
4,425,97097.0,160.0,NaN,diesel,silver,NaN,True,True,False,False,False,True,True,True,33400.0,2018-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4838,218 Gran Tourer,39743.0,110.0,NaN,diesel,black,NaN,False,True,False,NaN,False,False,True,False,14600.0,2018-08-01
4839,218 Active Tourer,49832.0,100.0,2015-06-01,diesel,grey,NaN,False,True,False,NaN,False,False,True,True,17500.0,2018-08-01
4840,218 Gran Tourer,19633.0,110.0,2015-10-01,diesel,grey,van,False,True,False,NaN,False,False,True,True,17000.0,2018-09-01
4841,218 Active Tourer,27920.0,110.0,2016-04-01,diesel,brown,van,True,True,False,False,False,False,True,True,22700.0,2018-09-01


In [66]:
df_bmw["asientos_traseros_plegables"].value_counts(dropna=False)

asientos_traseros_plegables
NaN      3391
False    1150
True      302
Name: count, dtype: int64

In [67]:
df_bmw[df_bmw["asientos_traseros_plegables"]].value_counts()

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [85]:

def separate_values(df, columna, value1 = True, value2 = False):
    first_condition_list = []
    second_condition_list = []
    no_value = []
    for index, row in df.iterrows():
        if row[f"{columna}"] == value1:
            first_condition_list.append({row["precio"], row["modelo"]})
        elif row[f"{columna}"] == value2:
            second_condition_list.append({row["precio"], row["modelo"]})
        else:
            no_value.append({row["precio"], row["modelo"]})      
    return first_condition_list, second_condition_list

In [104]:
lista_con_asientos, lista_sin_asientos = separate_values(df_bmw,"asientos_traseros_plegables")


In [107]:
# Convertir cada set a una lista ordenada (ej: strings primero)
lista_ordenada_con = [
    sorted(s, key=lambda x: isinstance(x, str), reverse=True) 
    for s in lista_con_asientos
]

df_asientos_true = pd.DataFrame(lista_ordenada_con, columns=["modelo", "precio"]) # df del modelo y precio de los TRUE


lista_ordenada_sin = [
    sorted(s, key=lambda x: isinstance(x, str), reverse=True) 
    for s in lista_sin_asientos
]

df_asientos_false = pd.DataFrame(lista_ordenada_sin, columns=["modelo", "precio"]) # df de el modelo y el precio de los FALSE


In [122]:
df_asientos_true_2 = df_asientos_true[df_asientos_true.duplicated()]
df_asientos_true_2.sort_values(ascending=False, by=["modelo","precio"])

,modelo,precio
269,X6,22000.0
274,X6,22000.0
247,X5,37800.0
285,X5,37400.0
253,X5,34500.0
262,X5,34500.0
288,X5,34200.0
291,X5,13700.0
237,X3,27300.0
283,X3,26800.0


In [120]:
df_asientos_false_2 = df_asientos_false[df_asientos_false.duplicated()]
df_asientos_false_2.sort_values(ascending=False, by=["modelo","precio"])

,modelo,precio
1136,X3,22000.0
1051,X3,18200.0
1100,X3,16500.0
1140,X3,15500.0
1064,X3,15200.0
...,...,...
604,116,8200.0
987,116,8000.0
995,116,8000.0
649,116,7900.0


In [123]:
df_asientos_false_2["asientos_plegables"] = False
df_asientos_true_2["asientos_plegables"] = True

/var/folders/97/rp671bk13hx384_fw0xl5grm0000gn/T/ipykernel_71686/1148979480.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_asientos_false_2["asientos_plegables"] = False
/var/folders/97/rp671bk13hx384_fw0xl5grm0000gn/T/ipykernel_71686/1148979480.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_asientos_true_2["asientos_plegables"] = True


In [128]:
df_asientos_not_null = pd.concat([df_asientos_true_2, df_asientos_false_2], axis=0)



In [50]:
df_asientos_not_null

NameError: name 'df_asientos_not_null' is not defined

In [49]:
corr = df_asientos_not_null.corr(numeric_only=True)
corr.style.background_gradient(cmap="coolwarm")

NameError: name 'df_asientos_not_null' is not defined

In [9]:
df_bmw["fecha_venta"] = pd.to_datetime(df_bmw["fecha_venta"])

In [10]:
df_bmw["fecha_venta"]

0      2018-01-01
1      2018-02-01
2      2018-02-01
3      2018-02-01
4      2018-04-01
          ...    
4838   2018-08-01
4839   2018-08-01
4840   2018-09-01
4841   2018-09-01
4842   2018-09-01
Name: fecha_venta, Length: 4843, dtype: datetime64[ns]

In [11]:
df_bmw[df_bmw.duplicated()].shape

(0, 18)